I have a generative function with a single variable but 2000 observations or I just want to use/apply it repeatedly, what do I do?

In [ ]:
import genjax
import jax
import jax.numpy as jnp
from genjax import bernoulli, gen


# First start by creating a simple generative function
@gen
def double_flip(p, q):
    v1 = bernoulli(p) @ "v1"
    v2 = bernoulli(q) @ "v2"
    return v1 + v2


# Now we can create a vectorized version that takes a batch of p values
# and calls the function for each value in the batch.
# The `in_axes` tell the `vmap_combinator` which arguments are
# mapped over, and which are not.
# The value `0` means we will map over this argument and `None` means we will not.
batched_double_flip = genjax.vmap_combinator(double_flip, in_axes=(0, None))

# Now we can use the batched version to generate a batch of samples
key = jax.random.PRNGKey(0)
size_of_batch = 20
# To do so, we have to create batched keys and p values
p = jax.random.uniform(key, (size_of_batch,))
q = 0.5
# We will run the generative function once for (p1, q), once for (p2, q), ...
traces = batched_double_flip.simulate(key, (p, q))
print(traces.get_retval())

# We can also use call it on (p1,q1), (p2,q2), ...
p = jax.random.uniform(key, (size_of_batch,))
q = jax.random.uniform(key, (size_of_batch,))
batched_double_flip_v2 = genjax.vmap_combinator(double_flip, in_axes=(0, 0))
traces = batched_double_flip_v2.simulate(key, (p, q))
print(traces.get_retval())

In [ ]:
# Note: We cannot batch different variables with different shapes
try:
    p = jax.random.uniform(key, (size_of_batch,))
    q = jax.random.uniform(key, (size_of_batch + 1,))
    traces = batched_double_flip_v2.simulate(key, (p, q))
    print(traces.get_retval())
except Exception:
    print(
        "Error: The batched version of the generative function is not working correctly"
    )

What about iterating `vmap`, e.g. if we want to apply a generative function acting on a pixel over a 2D space?

In [ ]:
key = jax.random.PRNGKey(42)
image = jnp.zeros([300, 500], dtype=jnp.float32)


# Generative function on one "pixel" value
@gen
def sample_pixel(pixel):
    new_pixel = genjax.normal(pixel, 1.0) @ "new_pixel"
    return new_pixel


tr = sample_pixel.simulate(key, (0.0,))
print("new_pixel:", tr.get_sample()["new_pixel"])

# Now what if we want to apply a generative function over a 2D space?
# We can use a nested `vmap_combinator`
sample_image = genjax.vmap_combinator(in_axes=(0,))(
    genjax.vmap_combinator(in_axes=(0,))(sample_pixel)
)

tr = sample_image.simulate(key, (image,))
# We can access the new_pixel value for each pixel in the image
print(tr.get_sample())
print("2D space choicemap:", tr.get_sample()[0, 0, "new_pixel"])
print("2D space choicemap:", tr.get_sample()[299, 499, "new_pixel"])

# Model wrapped in a bigger model
image = jnp.zeros([2, 3], dtype=jnp.float32)


@gen
def model(p):
    sampled_image = sample_image(image) @ "sampled_image"
    return sampled_image[0] + p


tr = model.simulate(key, (0.0,))
# We can use ellipsis to access the new_pixel value for each pixel in the image
print("sampled_image:", tr.get_sample()["sampled_image", ..., ..., "new_pixel"])
print()

# Alternatively, we can flatten the 2 dimensions into one and use a single `vmap_combinator`.
# This can be more efficient in some cases and usually has a faster compile time.
sample_image_flat = genjax.vmap_combinator(in_axes=(0,))(sample_pixel)
tr = sample_image_flat.simulate(key, (image.flatten(),))
# resize the sample to the original shape
out_image = tr.get_sample()[..., "new_pixel"].reshape(image.shape)
print("sampled_image:", out_image)

Oh but my iteration is actually over time, not space, i.e. I may want to reuse the same model by composing it with itself, e.g. for an HMM. 
For this, we can use the `scan` combinator.

In [ ]:
# Simple kernel for a Hidden Markov Model (HMM) example.
@gen
def hmm_kernel(x):
    z = genjax.normal(x, 1.0) @ "z"
    y = genjax.normal(z, 1.0) @ "y"
    return y


# Now we can create a function that runs the kernel multiple times
@genjax.scan_combinator(max_length=10)
@gen
def hmm(x, c):
    x1 = hmm_kernel(x) @ "x1"
    return x1, None


# Alternatively, we can directly create the same HMM model
@genjax.scan_combinator(max_length=10)
@gen
def hmm_v2(x, c):
    z = genjax.normal(x, 1.0) @ "z"
    y = genjax.normal(z, 1.0) @ "y"
    return y, None


key = jax.random.PRNGKey(42)
key, subkey = jax.random.split(key)
initial_x = 0.0
tr_1 = hmm.simulate(key, (initial_x, None))
print("Value of z at the beginning:", tr_1.get_sample()[0, "x1", "z"])
print("Value of y at the end:", tr_1.get_sample()[9, "x1", "y"])
print(tr_1.get_sample()[..., "x1", "z"])

tr_2 = hmm_v2.simulate(subkey, (initial_x, None))
print(tr_2.get_sample()[0, "z"])
print(tr_2.get_sample()[9, "y"])
print(tr_2.get_sample()[..., "z"])

In [ ]:
# Alternatively, we can call the generative function with a repeat combinator
# This will run the generative function multiple times on a single argument and return the results
@genjax.gen
def model(y):
    x = genjax.normal(y, 0.01) @ "x"
    y = genjax.normal(x, 0.01) @ "y"
    return y


key = jax.random.PRNGKey(0)
arg = 3.0
tr = model.repeat(n=10).simulate(key, (arg,))

print(tr.get_sample()[..., "x"])
print()
print(tr.get_retval())
print()

# It can be combined with vmap
sub_keys = jax.random.split(key, 3)
args = jnp.array([1.0, 2.0, 3.0, 4.0, 5.0])
n = 3
tr = jax.jit(jax.vmap(model.repeat(n=n).simulate, in_axes=(0, None)))(sub_keys, (args,))
print(tr.get_sample())
print()

# Note that it's running a computation |keys| * |args| * |n| times,
# i.e. 45 times in this case
print(tr.get_retval())